In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("result_seq2seq.csv")

In [3]:
data["answer"] = data["answer"].str.replace('_', '')

data

In [4]:
ans = data["answer"]
pred = data["predict"]

# 完全一致率の計算

In [5]:
cnt = 0
for i in range(len(ans)):
    if ans[i] == pred[i]:
        #print("ans:"+ans[i]+" pred:"+pred[i])
        cnt+=1

In [6]:
cnt

1262

In [13]:
same_percentage = (cnt/len(ans))*100

In [14]:
same_percentage

5.684684684684685

# 種類数の計算

In [9]:
col = []
for i in range(len(pred)):
    if pred[i] not in col:
        #print(pred[i])
        col.append(pred[i])

In [10]:
len(col)

49

# BLUEの計算

In [11]:
import nltk

In [12]:
nltk.translate.bleu_score.sentence_bleu(ans,pred)

0